In [46]:
from blackjack import BlackJack
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
game = BlackJack()

In [16]:
game.print_state()

7 Five False


In [128]:
def policy(state):
    (card_sum, dealer_card, usable_ace) = state
    if card_sum < 16:
        return "hit"
    elif card_sum < 20 and usable_ace:
        return "hit"
    else:
        return "stick"

In [149]:
#MC policy evaluation
#number of states is given by (21 - 12 + 1) * (Ace-King) * 2
total_returns = np.zeros((21-12+1, 13, 2))
total_visits = np.ones((21-12+1, 13, 2))
score = 0
iters = 500000
for i in range(500000):
    game = BlackJack()
    states = []
    while(not(game.game_ended)):
#         game.print_state()
        current_state = game.get_state()
        states.append(current_state)
        action = policy(game.get_state())
#         print("Action: {}".format(action))
        game.take_action(action)
#     game.print_player_hand()
#     game.print_dealer_hand()
#     print(game.game_outcome)
    
    score +=game.game_outcome
    for state in states:
        total_returns[state[0] - 12, state[1], 1 if state[2] else 0] += game.game_outcome
        total_visits[state[0] - 12, state[1], 1 if state[2] else 0] += 1

state_values = total_returns/total_visits;
# print(state_values)

In [150]:
print(score/iters)

-0.012624


#### fig = plt.figure(figsize=(30,10))

X = np.arange(1, 14) #Dealer showing card 
Y = np.arange(12, 22) #Player sum
X, Y = np.meshgrid(X, Y)

#Without usable ace
ax = fig.add_subplot(121, projection='3d')
ax.view_init(50, -45)
ax.plot_wireframe(X, Y, state_values[:,:,0], rstride=1, cstride=1, linewidth=1)
ax.set_title("No usable ace")
ax.set_xlabel("Dealer showing")
ax.set_ylabel("Player sum")

ax = fig.add_subplot(122, projection='3d')
ax.view_init(50, -45)
ax.plot_wireframe(X, Y, state_values[:,:,1], rstride=1, cstride=1, linewidth=1, color="g")
ax.set_title("Usable ace")
ax.set_xlabel("Dealer showing")
ax.set_ylabel("Player sum")
